# Challenge 1 - ret2win
by ROP Emporium



## ROP Emporium
This is a great place to learn about Return Oriented Programming, one of the sexier buffer overflow techniques used today. This is a CTF site where you attempt to exploit a binary using ROP in order to read out flag.txt

<a href="https://ropemporium.com/" target="_blank"><img src="https://www.google.com/url?sa=i&url=https%3A%2F%2Fwww.reddit.com%2Fr%2FExploitDev%2Fcomments%2Fhxom8x%2Frop_emporium_pwning_mips%2F&psig=AOvVaw30vCKW9fAZhTe9NLT8Ta68&ust=1639504051768000&source=images&cd=vfe&ved=0CAsQjRxqFwoTCNCTrt6q4fQCFQAAAAAdAAAAABAD" 
alt="FOOL, THE IMAGE IS GONE" width="240" height="180" border="10" /></a>

ROP Emporium offers their binaries in 4 architectural flavors, to start out I'll be using x86_64 (very basic I know) but I plan to run through the gauntlet again at least one more time to learn about ARM Pointer Auth Codes.